# Segmenting and Clustering Neighborhoods in Toronto

## Section 1 - Creating a Toronto dataframe

In [3]:
!pip install lxml
!pip install beautifulsoup4
!pip install geopy
!pip install folium
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes

In [4]:
from lxml import html
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [5]:
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(url)
print(response.status_code)

200


In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
torontotable=soup.find('table',{'class':"wikitable"})

In [7]:
df=pd.read_html(str(torontotable))
# convert list to dataframe
df_neigh=pd.DataFrame(df[0])
print(df_neigh.head())
print(df_neigh.info())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postcode       288 non-null    object
 1   Borough        288 non-null    object
 2   Neighbourhood  288 non-null    object
dtypes: object(3)
memory usage: 6.9+ KB
None


In [8]:
df_neigh = df_neigh[df_neigh.Borough != 'Not assigned']

In [9]:
df_neigh = df_neigh.groupby(by=['Postcode','Borough']).agg(lambda x: ', '.join(x)) 
df_neigh.reset_index(level=['Postcode','Borough'], inplace=True) 
df_neigh

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [12]:
df_neigh['Neighbourhood'].replace(df_neigh['Borough'],'Not assigned')
df_neigh = df_neigh.rename(columns={"Postcode": "Postalcode", "Neighbourhood": "Neighborhood"})
df_neigh

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [13]:
df_neigh.shape

(103, 3)

## Section 2 - Getting Lat and Long cordinates

In [14]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_coord = df_coord.rename(columns={"Postal Code": "Postalcode"})
df_coord

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [16]:
df_table = pd.merge(df_neigh, df_coord, on='Postalcode')
df_table

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## Section 3 - Neighborhood Clustering

In [17]:
toronto_data = df_table[df_table['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
CLIENT_ID = 'client_id=243GFCEQ5JWNK5YSQ02KS4AYLHU1IKHQGZDM0V0PBZYUCTRS' # your Foursquare ID
CLIENT_SECRET = 'client_secret=SI0OX0YRTODV3SU1HVZMFXUO2XCOQJUVSB4IYMF13U3E2LL1' # your Foursquare Secret
access_token = '53LFMOFMXER1TFUCUVDLPCVMEMRXYPHY1NJS2WWBRS0VOYLL'
search_query=''
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng, 
            access_token,
            VERSION, 
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(2109, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,The Beaches,43.676357,-79.293031,Domino's Pizza,43.679058,-79.297382,Pizza Place


In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.01,0.000000,0.00
1,Berczy Park,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.00,0.000000,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.029412,0.00,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.043478,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.00,0.00,0.052632,0.052632,0.052632,0.105263,0.157895,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00
5,"Cabbagetown, St. James Town",0.017857,0.017857,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00
6,Central Bay Street,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.010000,0.000000,0.00,0.050000,0.000000,0.030000,0.01,0.000000,0.00
8,Christie,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00
9,Church and Wellesley,0.030000,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.010000,0.01


In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Gym,Cosmetics Shop,Thai Restaurant,Sushi Restaurant,Steakhouse,Asian Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Sandwich Place,Pub,Italian Restaurant,Vegetarian / Vegan Restaurant,Steakhouse,Bistro,Pharmacy
2,"Brockton, Exhibition Place, Parkdale Village",Café,Convenience Store,Coffee Shop,Gym,Breakfast Spot,Sandwich Place,Gym / Fitness Center,Bus Stop,Bar,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Gym / Fitness Center,Light Rail Station,Fast Food Restaurant,Burrito Place,Butcher,Comic Shop,Farmers Market,Garden,Garden Center,Martial Arts School
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Bar,Boat or Ferry,Sculpture Garden,Boutique,Coffee Shop,Airport Gate


In [58]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 7, 7, 7, 7, 0, 7, 7, 0], dtype=int32)

In [59]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,7,Pizza Place,Home Service,Asian Restaurant,Pub,Trail,Coffee Shop,Park,Health Food Store,Miscellaneous Shop,Middle Eastern Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,7,Greek Restaurant,Spa,Pharmacy,Sushi Restaurant,Coffee Shop,Ice Cream Shop,Intersection,Italian Restaurant,Frozen Yogurt Shop,Pub
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,7,Sandwich Place,Pizza Place,Park,Fast Food Restaurant,Mobile Phone Shop,Steakhouse,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Board Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,7,Coffee Shop,Café,Pizza Place,Furniture / Home Store,Bakery,Gastropub,American Restaurant,Clothing Store,Brewery,Convenience Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Gym / Fitness Center,Park,Bus Line,Swim School,Lawyer,Business Service,Construction & Landscaping,Photography Studio,Mobile Phone Shop,Modern European Restaurant


In [60]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Central Toronto,0,Dessert Shop,Café,Sandwich Place,Gym,Sushi Restaurant,Coffee Shop,Pizza Place,Thai Restaurant,Italian Restaurant,Discount Store
52,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Gay Bar,Japanese Restaurant,Yoga Studio,Restaurant,Burrito Place,Mediterranean Restaurant,Men's Store,Mexican Restaurant
54,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Diner,Burger Joint
55,Downtown Toronto,0,Café,Coffee Shop,Italian Restaurant,Hotel,Restaurant,Breakfast Spot,Seafood Restaurant,Bakery,Cosmetics Shop,Gastropub
56,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Sandwich Place,Pub,Italian Restaurant,Vegetarian / Vegan Restaurant,Steakhouse,Bistro,Pharmacy
57,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Thai Restaurant,Ice Cream Shop,Italian Restaurant,Middle Eastern Restaurant,Pharmacy
58,Downtown Toronto,0,Coffee Shop,Café,American Restaurant,Gym,Cosmetics Shop,Thai Restaurant,Sushi Restaurant,Steakhouse,Asian Restaurant,Hotel
59,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Restaurant,Fried Chicken Joint,Italian Restaurant,Scenic Lookout,Brewery,Sporting Goods Shop
60,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Steakhouse,Beer Bar,Gastropub
61,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gym,Japanese Restaurant,American Restaurant,Cocktail Bar,Deli / Bodega


In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,1,Park,Playground,Trail,Modern European Restaurant,Museum,Movie Theater,Motorcycle Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,2,Beer Store,Park,Tennis Court,Playground,Pharmacy,Music Venue,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,3,Gym / Fitness Center,Park,Bus Line,Swim School,Lawyer,Business Service,Construction & Landscaping,Photography Studio,Mobile Phone Shop,Modern European Restaurant


In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,4,Park,Trail,Sushi Restaurant,Bus Line,Jewelry Store,Health & Beauty Service,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Middle Eastern Restaurant


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Central Toronto,5,Hotel,Electronics Store,Park,Supplement Shop,Department Store,Dance Studio,Food & Drink Shop,Breakfast Spot,Sandwich Place,Gym


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,6,Home Service,Ice Cream Shop,Health & Beauty Service,Pool,Fast Food Restaurant,Yoga Studio,Music Venue,Museum,Movie Theater,Motorcycle Shop


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,7,Pizza Place,Home Service,Asian Restaurant,Pub,Trail,Coffee Shop,Park,Health Food Store,Miscellaneous Shop,Middle Eastern Restaurant
41,East Toronto,7,Greek Restaurant,Spa,Pharmacy,Sushi Restaurant,Coffee Shop,Ice Cream Shop,Intersection,Italian Restaurant,Frozen Yogurt Shop,Pub
42,East Toronto,7,Sandwich Place,Pizza Place,Park,Fast Food Restaurant,Mobile Phone Shop,Steakhouse,Liquor Store,Food & Drink Shop,Fish & Chips Shop,Board Shop
43,East Toronto,7,Coffee Shop,Café,Pizza Place,Furniture / Home Store,Bakery,Gastropub,American Restaurant,Clothing Store,Brewery,Convenience Store
46,Central Toronto,7,Clothing Store,Sporting Goods Shop,Spa,Coffee Shop,Cosmetics Shop,Salon / Barbershop,Rental Car Location,Yoga Studio,Italian Restaurant,Diner
49,Central Toronto,7,Coffee Shop,Light Rail Station,Athletics & Sports,Sushi Restaurant,Vietnamese Restaurant,Bank,Bagel Shop,Fried Chicken Joint,Skating Rink,Supermarket
51,Downtown Toronto,7,Coffee Shop,Pizza Place,Café,Bakery,Italian Restaurant,Liquor Store,Market,Gourmet Shop,Pharmacy,Pet Store
53,Downtown Toronto,7,Coffee Shop,Pub,Park,Furniture / Home Store,Art Gallery,Bakery,Theater,Café,Performing Arts Venue,Bus Stop
65,Central Toronto,7,Café,Coffee Shop,Sandwich Place,Grocery Store,Furniture / Home Store,Donut Shop,Liquor Store,Martial Arts School,Metro Station,Convenience Store
67,Downtown Toronto,7,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Bakery,Caribbean Restaurant,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant
